In [1]:
"""
Corpus example taking from PhD thesis

abbac
abccba
bbac
cba """
D=[["a","b","b","a","c"],["a","b","c","c","b","a"],["b","b","a","c"],["c","b","a"]]
N=8

In [2]:
def ngrams(input, n):
  if n<=0:
    return {}
  output = {}
  for i in range(len(input)-n+1):
    g = ' '.join(input[i:i+n])
    output.setdefault(g, 0)
    output[g] += 1
  return output 

def add(x,y):
    return { k: x.get(k, 0) + y.get(k, 0) for k in set(x) | set(y) }

#TODO how to make this in a way to not compute it over and over again
def l(m,rho):
    if m<1 or rho <=0 or rho>=1:
        raise ("argument in wrong range, m should be an int bigger than 0 and rho a real between 0 and 1. rho was: "+str(rho) + " m was: " + str(m))
    return rho*pow(1-rho,m-1)

In [3]:
absoluteCounts = {i:{} for i in range(0,N)}
for s in D:
    for i in range(0,N):
        res = ngrams(s,i)
        absoluteCounts[i] = add(absoluteCounts[i],res)

totalCounts = {i:sum(absoluteCounts[i].values()) for i in range(0,N)}
        
for i in range(0,N):
    print i, totalCounts[i], absoluteCounts[i]


0 0 {}
1 18 {'a': 6, 'c': 5, 'b': 7}
2 14 {'a b': 2, 'a c': 2, 'b a': 4, 'b c': 1, 'b b': 2, 'c b': 2, 'c c': 1}
3 10 {'a b b': 1, 'a b c': 1, 'b a c': 2, 'b b a': 2, 'c b a': 2, 'c c b': 1, 'b c c': 1}
4 6 {'a b c c': 1, 'b c c b': 1, 'c c b a': 1, 'b b a c': 2, 'a b b a': 1}
5 3 {'a b c c b': 1, 'b c c b a': 1, 'a b b a c': 1}
6 1 {'a b c c b a': 1}
7 0 {}


In [4]:
#TODO: make this more general
ngramModels={i:{k:float(v)/totalCounts[i] for k,v in absoluteCounts[i].iteritems()} for i in range(0,N)}

In [5]:
for i in range(0,N):
    print i, totalCounts[i], ngramModels[i]

0 0 {}
1 18 {'a': 0.3333333333333333, 'c': 0.2777777777777778, 'b': 0.3888888888888889}
2 14 {'a b': 0.14285714285714285, 'a c': 0.14285714285714285, 'b a': 0.2857142857142857, 'b c': 0.07142857142857142, 'b b': 0.14285714285714285, 'c b': 0.14285714285714285, 'c c': 0.07142857142857142}
3 10 {'a b b': 0.1, 'a b c': 0.1, 'b a c': 0.2, 'b b a': 0.2, 'c b a': 0.2, 'b c c': 0.1, 'c c b': 0.1}
4 6 {'c c b a': 0.16666666666666666, 'a b b a': 0.16666666666666666, 'a b c c': 0.16666666666666666, 'b b a c': 0.3333333333333333, 'b c c b': 0.16666666666666666}
5 3 {'a b c c b': 0.3333333333333333, 'b c c b a': 0.3333333333333333, 'a b b a c': 0.3333333333333333}
6 1 {'a b c c b a': 1.0}
7 0 {}


In [21]:
from functools import partial
import itertools

W="abc"   


def buildProducts(W,n):
    return map(" ".join,itertools.product(W, repeat= n))

def buildVocabulary(D):
    res ={}
    for sentence in D:
        #TODO: introduce caching via dictionary
        tmp = ngrams(sentence,1)
        res = add(tmp,res)
    print res,res.keys()
    return res.keys()

def buildAbsoluteCounter(D,n):
    W = buildVocabulary(D)
    W_n = buildProducts(W,n)
    
    res = {}
    def absoluteCounts(s):
        if s in res:
            return res[s]
        if s in W_n:
            print "calc shit"
            for sentence in D:
                #TODO: introduce caching via dictionary
                tmp = ngrams(sentence,n)
                for k in set(res) | set(tmp):
                    res[k]=res.get(k, 0) + tmp.get(k, 0)
            if s in res:
                return res[s]
            return 0
    return absoluteCounts
    
def memo(f):
    d = {}
    def func(arg):
        if arg not in d:
            d[arg]=f(arg)
        return d[arg]
    return func

def ngram(n):
    def totalSum(arg,baseSet):
        return sum([arg(el) for el in baseSet])

    def c(s):
        res ={}
        for sentence in D:
            #TODO: introduce caching via dictionary
            tmp = ngrams(sentence,n)
            for k in set(res) | set(tmp):
                res[k]=res.get(k, 0) + tmp.get(k, 0)
        if s in res:
            return res[s]
        return 0

    def uniform(s):
        return 1

    def estimator(f,s):
        return float(f(s))/totalSum(f,buildProducts(W,n))    
    return {"a":c,"u":uniform,"ae":partial(estimator, c), "ue":partial(estimator, uniform)}

In [22]:
print ngram(3)["ae"]("a b b")
c3 = buildAbsoluteCounter(D,3)

print ngram(3)["ue"]("a b b")
print buildVocabulary(D)
print buildProducts(buildVocabulary(D),2)

c3("a b b")

0.1
{'a': 6, 'c': 5, 'b': 7} ['a', 'c', 'b']
0.037037037037
{'a': 6, 'c': 5, 'b': 7} ['a', 'c', 'b']
['a', 'c', 'b']
{'a': 6, 'c': 5, 'b': 7} ['a', 'c', 'b']
['a a', 'a c', 'a b', 'c a', 'c c', 'c b', 'b a', 'b c', 'b b']
calc shit


1

In [ ]:

def buildStuff(D,n):
res = {}
    def c(s):
        if not s in res:
            res = myFunc(100)
            return 0
        else:
            return res[s]
    return c
ok, modulo formatierung
anyway, die zuweisung im c macht Probleme:

res = myFunc(100)
das hier erzeugt ne neue variable res die die andere überschattet
und sowas hattest du auch im code wenn ich mich richtig erinnere

In [189]:
for s in buildProducts(buildVocabulary(D),3):
    print c3(s)
    
for s in buildProducts(buildVocabulary(D),3):
    print c3(s)
    

{'a': 6, 'c': 5, 'b': 7} ['a', 'c', 'b']
calc shit
0
calc shit
0
calc shit
0
calc shit
0
calc shit
0
calc shit
0
calc shit
0
calc shit
1
1
calc shit
0
calc shit
0
calc shit
0
calc shit
0
calc shit
0
calc shit
1
calc shit
2
calc shit
0
calc shit
0
calc shit
0
calc shit
2
calc shit
0
calc shit
0
calc shit
1
calc shit
0
calc shit
2
calc shit
0
calc shit
0
{'a': 6, 'c': 5, 'b': 7} ['a', 'c', 'b']
0
0
0
0
0
0
0
1
1
0
0
0
0
0
1
2
0
0
0
2
0
0
1
0
2
0
0


In [150]:
def foo(arg):
    print "call"
    return arg* arg
def bar(arg):
    print "call"
    return arg* arg * arg

times = memo(bar)
cachedsquare = memo(foo)

In [151]:
print cachedsquare(2)
print cachedsquare(2)
print cachedsquare(2)

print times(2)
print times(2)
times = memo(bar)
print times(2)


call
4
4
4
call
8
8
call
8


In [167]:
#print order(3)["e"]("a b b",order(3)["a"])
import itertools
print map(" ".join,itertools.product("abc",repeat = 2))

['a a', 'a b', 'a c', 'b a', 'b b', 'b c', 'c a', 'c b', 'c c']


In [ ]:
def c(n,s):
    res ={}
    for sentence in D:
        tmp = ngrams(sentence,n)
        res = add(tmp,res)
    return res[s]
print c(2,)

In [44]:
print l(5,0.5)

def queryNgramModel(query):
    words = query.split(" ")
    m = len(words)
    return ngrams[m][query]*l(m,0.5)
    

0.03125


In [45]:
print queryNgramModel("a b")

TypeError: 'function' object has no attribute '__getitem__'